In [ ]:
from pokemontcgsdk import Card, Set, RestClient
from PIL import Image
from io import BytesIO
import requests
import pandas as pd
from sentence_transformers import SentenceTransformer
from Utils.cardParser import extractCore
import time
from requests.exceptions import ConnectionError, Timeout, HTTPError
import os
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
api_key = "api-key"
RestClient.configure(api_key)

In [ ]:
cards = Card.all()#fetch all cards from the sdk

In [ ]:
#Parse the objects to a dataframe each card as a row
df = pd.DataFrame(columns=["id", "name", "rarity", "superType", "set_id", "img_path_large", "img_path_small", "card_class_dump"])

for card in tqdm(cards, total = len(cards), desc="Processing data"):
    df.loc[len(df)] = extractCore(card)

In [ ]:
model = SentenceTransformer("clip-ViT-B-16")

In [ ]:
embeddings = []

In [ ]:
for index, c in tqdm(df.iloc.iterrows(), total=len(df), desc="Processing Rows"):

    img = requests.get(c["img_path_small"]).content
    img = Image.open(BytesIO(img))
    
    emb = model.encode(img, normalize_embeddings=True)
    embeddings.append(emb)

In [ ]:
df["embedding"] = embeddings

In [ ]:
df.to_parquet("MASTER_SET_b16.parquet")